In [1]:
import pyterrier as pt
if not pt.started():
  pt.init()

     |████████████████████████████████| 126 kB 13.3 MB/s 
     |████████████████████████████████| 95 kB 3.5 MB/s 
     |████████████████████████████████| 1.1 MB 39.6 MB/s 
     |████████████████████████████████| 69 kB 7.2 MB/s 
     |████████████████████████████████| 222 kB 48.0 MB/s 
     |████████████████████████████████| 1.8 MB 42.7 MB/s 
     |████████████████████████████████| 596 kB 46.8 MB/s 
     |████████████████████████████████| 294 kB 46.2 MB/s 
     |████████████████████████████████| 6.3 MB 38.2 MB/s 
     |████████████████████████████████| 72 kB 1.4 MB/s 
     |████████████████████████████████| 291 kB 37.9 MB/s 
     |████████████████████████████████| 45 kB 2.9 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.7.1-py3-none-any.whl size=102452 sha256=5d0cc5c788273b41ce0a2c3898eab277be7c3cb99da8cbe956f25cfedbeeb88d
  Stored in directory: /root/.cache/pip/wheels/c0/9a/c8/1c2d9ec6a1494bb54f47e0d2627b5ed7b2de704b66723d3417
  Created wheel for ir-measures: filen

In [3]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/workspace/corpus.csv')

In [4]:
pd_indexer = pt.DFIndexer("./pd_index")
indexref = pd_indexer.index(df["text"], df["docno"])

15:16:22.904 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (Sdbc891ec-A55b98352) - further warnings are suppressed
15:20:52.176 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 57 empty documents
15:20:52.692 [main] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key Saeff5ad8-Afba459ff is not unique: 160773,160770
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:346)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:369)


In [5]:
from xml.etree import ElementTree
tree = ElementTree.parse('/content/drive/MyDrive/workspace/topics.xml')
data = tree.getroot()
titles = []
title_ids = []
for topic in data:
    titles.append(topic.find('title').text.lower()[:-1])
    num = topic.find('number').text
    title_ids.append(num)


In [6]:
topics = pd.DataFrame(titles,columns = ['query'])
topics['qid'] = topics.index + 51
columns_titles = ["qid","query"]
topics=topics.reindex(columns=columns_titles)

In [7]:
index = pt.IndexFactory.of(indexref)
retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"})
res=retr.transform(topics)
qrels_path=("/content/drive/MyDrive/workspace/qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

In [8]:
res

,qid,docid,docno,rank,score,query
0,51,297468,S885c6b4f-Ad3259e81,0,15.174829,do we need sex education in schools
1,51,330918,S6353b9ac-A9cb2062c,1,14.984042,do we need sex education in schools
2,51,222529,S428a8bea-Aba4c7b80,2,14.776028,do we need sex education in schools
3,51,279241,Sd32774d-A5bbc95,3,14.715350,do we need sex education in schools
4,51,330916,S6353b9ac-A2df6348e,4,14.638091,do we need sex education in schools
...,...,...,...,...,...,...
49995,100,256729,S2ea519c-Aed35ffc8,995,3.581751,do we need cash
49996,100,29220,Sa68e9864-Af79ef325,996,3.580354,do we need cash
49997,100,231260,Sfb30ce24-Aa94950e7,997,3.580354,do we need cash
49998,100,294707,S78819fc5-Ae076e471,998,3.576581,do we need cash


In [9]:
eval = pt.Utils.evaluate(res,qrels,metrics=["ndcg_cut_5"])
eval

{'ndcg_cut_5': 0.5527540851759042}

In [10]:
import os
if not os.path.exists("uncased_L-12_H-768_A-12.zip"):
  !wget https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-12_H-768_A-12.zip
  !unzip uncased_L-12_H-768_A-12.zip
  !mkdir -p bert-base-uncased
  !mv vocab.txt bert_* bert-base-uncased/
%tensorflow_version 1.x


if not os.path.exists("marco.zip"):
  !wget http://boston.lti.cs.cmu.edu/appendices/arXiv2019-DeepCT-Zhuyun-Dai/outputs/marco.zip
  !unzip marco.zip

import pyterrier_deepct
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
assert tf.__version__.startswith("1"), "TF 1 is required by DeepCT; on Colab, use %tensorflow_version 1.x"
tf.logging.set_verbosity(tf.logging.ERROR)
deepct = pyterrier_deepct.DeepCTTransformer("bert-base-uncased/bert_config.json", "marco/model.ckpt-65816")

--2021-11-17 15:21:12--  https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 66.102.1.128, 142.251.5.128, 74.125.133.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|66.102.1.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 408102251 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 389.20M  36.9MB/s    in 7.1s    

2021-11-17 15:21:19 (54.6 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [408102251/408102251]

Archive:  uncased_L-12_H-768_A-12.zip
  inflating: bert_model.ckpt.data-00000-of-00001  
  inflating: bert_config.json        
  inflating: vocab.txt               
  inflating: bert_model.ckpt.index   
TensorFlow 1.x selected.
  Cloning https://github.com/terrier-org/pyterrier.git to /tmp/pip-install-ipdnorhn/python-terrier_6d827037dad84db3b50ff746884b5384
  Running command git c

In [21]:
super_df = df.copy()
xd = super_df.head(500).copy()

In [22]:
with tf.device('/device:GPU:0'):
  deepct.transform(xd)

In [23]:
with tf.device('/device:GPU:0'):
  super_df = deepct.transform(super_df)

In [24]:
super_df.to_csv(index=False, path_or_buf='/content/drive/MyDrive/workspace/superdf.csv')

In [25]:
super_df

,docno,text
0,S1f6b58eb-A5c530110,hair hair hair hair hair hair hair hair hair h...
1,S1a9db4fc-Acc4206f5,un un un un un un un un un un un un un un un u...
2,S1a924cd4-A994e77ca,madam october october october october october ...
3,S1a3be6ac-A873cd7aa,frederick frederick frederick frederick freder...
4,S1beff81f-A1206f3ee,continued existence increase increase increase...
...,...,...
338615,Sca72da7d-A5cd811ac,lifeisgood lifeisgood debate clarifying meant ...
338616,Sca72da7d-Af008e63e,great pleasure pleasure debate debate debate d...
338617,Sca72da7d-Ac77ea7a1,forfeit forfeit forfeit forfeit forfeit forfei...
338618,Sca72da7d-Ae83ab999,opponent instigating instigating instigating i...


In [27]:
pd_indexer = pt.DFIndexer("./pd_super_index")
indexref = pd_indexer.index(super_df["text"], super_df["docno"])

17:54:33.344 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (S5a823806-Afabe27fa) - further warnings are suppressed
17:58:43.817 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 57 empty documents
17:58:44.240 [main] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key Saeff5ad8-Afba459ff is not unique: 160773,160770
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:346)
	at org.terrier.structures.indexing.Indexer.index(Indexer.java:369)


In [28]:
from xml.etree import ElementTree
tree = ElementTree.parse('/content/drive/MyDrive/workspace/topics.xml')
data = tree.getroot()
titles = []
title_ids = []
for topic in data:
    titles.append(topic.find('title').text.lower()[:-1])
    num = topic.find('number').text
    title_ids.append(num)

In [29]:
topics = pd.DataFrame(titles,columns = ['query'])
topics['qid'] = topics.index + 51
columns_titles = ["qid","query"]
topics=topics.reindex(columns=columns_titles)

In [30]:
index = pt.IndexFactory.of(indexref)
retr = pt.BatchRetrieve(index, controls = {"wmodel": "TF_IDF"})
res=retr.transform(topics)
qrels_path=("/content/drive/MyDrive/workspace/qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

In [31]:
res

,qid,docid,docno,rank,score,query
0,51,7966,Sb8c9af14-Ad2c4a6cf,0,23.320777,do we need sex education in schools
1,51,217824,S19dd934-A4cd5a64,1,23.174514,do we need sex education in schools
2,51,279241,Sd32774d-A5bbc95,2,21.680580,do we need sex education in schools
3,51,4205,Sf27da082-A442fab1b,3,21.556570,do we need sex education in schools
4,51,225945,Sb7c38e4f-A5d575a86,4,21.166800,do we need sex education in schools
...,...,...,...,...,...,...
49858,100,113532,S53107d7d-Af2bbe92c,995,5.509719,do we need cash
49859,100,24158,S9233af8e-A28044ff9,996,5.509667,do we need cash
49860,100,143266,S52db8af-A945b8980,997,5.509229,do we need cash
49861,100,13205,S2d7a6fbd-Ad1569014,998,5.508590,do we need cash


In [32]:
eval = pt.Utils.evaluate(res,qrels,metrics=["ndcg_cut_5"])
eval

{'ndcg_cut_5': 0.30676936612706834}